<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/Copia_de_PASPAS_ANON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers torch
import sqlite3
import pandas as pd
import re
import time
from transformers import pipeline, AutoTokenizer
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import torch # Import the torch module

# Cargar modelo NER y tokenizador
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-spanish-cased-finetuned-ner")


def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    # Tokenizar el texto con truncamiento y padding
    inputs = tokenizer(texto, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Realizar la inferencia
    with torch.no_grad(): # Deshabilitar el cálculo de gradientes para la inferencia
        outputs = ner_model.model(**inputs)

    # Obtener las predicciones
    predictions = ner_model.postprocess(outputs, aggregation_strategy=ner_model.aggregation_strategy)

    entidades = []
    for ent in predictions:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajustar límites de la entidad
            while start > 0 and texto[start-1] not in (' ', '\n', '\t'):
                start -= 1
            while end < len(texto) and texto[end] not in (' ', '\n', '\t'):
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades

def reemplazar_nombres(texto):
    """Reemplaza nombres detectados por una marca genérica."""
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = "[NOMBRE]"
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto

def procesar_fila(row):
    """Procesa una fila aplicando anonimización de nombres."""
    row["body"] = reemplazar_nombres(row["body"])
    return row


with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Cargar datos del archivo Excel a un DataFrame
    df_excel = pd.read_excel("MOSTRA_1.xlsx")

    # Crear la tabla 'mi_tabla' a partir del DataFrame
    df_excel.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df = pd.read_sql("SELECT id, body FROM mi_tabla", conn)

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df.to_dict(orient="records")), total=len(df)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")

ERROR: Operation cancelled by user


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
  0%|          | 0/6943 [00:00<?, ?it/s]


AttributeError: 'TokenClassificationPipeline' object has no attribute 'aggregation_strategy'

In [14]:
!pip install transformers torch
import sqlite3
import pandas as pd
import re
import time
from transformers import pipeline, AutoTokenizer
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import torch  # Import the torch module

# Cargar modelo NER y tokenizador
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    entidades = []

    try:
        predictions = ner_model(texto)
        for ent in predictions:
            if ent['entity_group'] == 'PER':
                start, end = ent['start'], ent['end']
                original = texto[start:end].strip()

                # Ajustar los límites para capturar nombres completos
                while start > 0 and texto[start-1] not in (' ', '\n', '\t'):
                    start -= 1
                while end < len(texto) and texto[end] not in (' ', '\n', '\t'):
                    end += 1

                entidades.append((start, end, original))
    except Exception as e:
        print(f"Error en NER: {e}")

    return entidades

def reemplazar_nombres(texto):
    """Reemplaza nombres detectados por una marca genérica."""
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = "[NOMBRE]"
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto

def procesar_fila(row):
    """Procesa una fila aplicando anonimización de nombres."""
    row["body"] = reemplazar_nombres(row["body"])
    return row

# Conectar con SQLite
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Cargar datos del archivo Excel a un DataFrame
    df_excel = pd.read_excel("MOSTRA_1.xlsx")

    # Crear la tabla 'mi_tabla' a partir del DataFrame
    df_excel.to_sql("mi_tabla", conn, if_exists="replace", index=False)
    conn.commit()  # Asegurar que los cambios se guarden

    # Extraer solo columnas relevantes
    df = pd.read_sql("SELECT id, body FROM mi_tabla", conn)

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(max(cpu_count() - 1, 1)) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df.to_dict(orient="records")), total=len(df)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)
    conn.commit()  # Confirmar los cambios

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
 42%|████▏     | 2942/6943 [06:36<13:06,  5.09it/s]

Error en NER: The size of tensor a (630) must match the size of tensor b (512) at non-singleton dimension 1


100%|██████████| 6943/6943 [15:24<00:00,  7.51it/s]

Procesamiento completado en 925.09 segundos.


In [15]:
# prompt: print all the columns of the db

import sqlite3
import pandas as pd

with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM PRAGMA_TABLE_INFO('mi_tabla')")
    columns = cursor.fetchall()
    print("Columnas de la tabla 'mi_tabla':")
    for column in columns:
column[0]


IndentationError: expected an indented block after 'for' statement on line 11 (<ipython-input-15-b3ddffa83698>, line 12)